<a href="https://colab.research.google.com/github/caqq07/ZAXAZCXCACAXCAXCA/blob/main/ImageGen2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q invisible_watermark transformers accelerate safetensors diffusers controlnet_aux==0.0.7 xformers mediapy

# Inference with pipelines
- For base SDXL

In [ ]:
import torch
from diffusers import StableDiffusionXLImg2ImgPipeline, DiffusionPipeline, KDPM2AncestralDiscreteScheduler, StableDiffusionXLPipeline, AutoencoderKL
import gc
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display


In [ ]:

from diffusers import StableDiffusionXLPipeline
import torch

model_base = "femboysLover/blue_pencil-fp16-XL"
v_autoencoder = "madebyollin/sdxl-vae-fp16-fix"

vae = AutoencoderKL.from_pretrained(v_autoencoder, torch_dtype=torch.float16)


pipe = StableDiffusionXLPipeline.from_pretrained(
    model_base,
    torch_dtype=torch.float16,
    vae=vae,
    add_watermarker=False, # no watermarker
    )

pipe.safety_checker = None

pipe.to("cuda")

- Refiner of SDXL.

In [ ]:
model_refiner = "stabilityai/stable-diffusion-xl-refiner-1.0"

pipe_refiner = StableDiffusionXLImg2ImgPipeline.from_pretrained(
    model_refiner,
    torch_dtype=torch.float16,
    use_safetensors=True,
    variant="fp16",
    add_watermarker=False, # no watermarker
    )

#pipe_refiner.to("cuda")
pipe_refiner.enable_model_cpu_offload()

In [ ]:
prompt = "(((hdr, masterpiece, highest resolution, best quality, beautiful, raw image))), (((extremely detailed, rendered))), female russian, white, ((age 25)), with Red hair, long face,  naked boobs,, ass view behind shot, Hair Style: curly, long, blue, maid outfit, In Istanbul, ((full body))"
prompt2 = "(((hdr, masterpiece, highest resolution, best quality, beautiful, raw image))), (((extremely detailed, rendered))), female russian, white, ((age 25)), with Red hair, long face,  naked boobs,, ass view behind shot, Hair Style: curly, long, blue, maid outfit, In Istanbul, naked boobs, ((full body))"
negative_prompt = "longbody, lowres, bad anatomy, bad hands, missing fingers, pubic hair,extra digit, fewer digits, cropped, worst quality, low quality"
negative_prompt2 = "longbody, lowres, bad anatomy, bad hands, missing fingers, pubic hair,extra digit, fewer digits, cropped, worst quality, low quality"

image_base_latent = pipe(
    prompt=prompt,
    prompt_2=prompt2,
    negative_prompt=negative_prompt,
    negative_prompt_2=negative_prompt2,
    guidance_scale=7.0,
    height=1344,
    width=768,
    num_inference_steps=25,
    output_type="latent",
    denoising_end=0.8 # Cut the base denoising in 80%
    ).images[0]
print(image_base_latent.shape)

gc.collect()
torch.cuda.empty_cache()

image_refiner = pipe_refiner(
    prompt=prompt,
    prompt_2=prompt2,
    negative_prompt=negative_prompt,
    negative_prompt_2=negative_prompt2,
    image=image_base_latent,
    num_inference_steps=25,
    height=1344,
    width=768,
    strength=0.3,
    denoising_start=0.8
    ).images[0]

gc.collect()
torch.cuda.empty_cache()

In [ ]:
image_refiner